## 自己构建的网络

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_pre
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_pre
from keras.preprocessing import image   
from keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py
import common
import pickle

Using TensorFlow backend.


## 对图片进行分类

In [2]:
common.divide_images()

In [3]:
models_and_features = {}
models_and_test_features = {}

In [4]:
def batch_extract_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg16_pre)
    models_and_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), vgg19_pre)
    models_and_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (224, 224), resnet50_pre)
    models_and_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), inceptionV3_pre)
    models_and_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_features(base_model, (299, 299), xception_pre)
    models_and_features['xception'] = features_name

In [5]:
def batch_extract_test_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), vgg16_pre)
    models_and_test_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), vgg19_pre)
    models_and_test_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (224, 224), resnet50_pre)
    models_and_test_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (299, 299), inceptionV3_pre)
    models_and_test_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_test_features(base_model, (299, 299), xception_pre)
    models_and_test_features['xception'] = features_name

In [6]:
batch_extract_features()

Found 19957 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.
249/249 [==============================] - 42s 167ms/step
Found 19957 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.
249/249 [==============================] - 49s 198ms/step
Found 19957 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.
249/249 [==============================] - 42s 171ms/step
Found 19957 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.
249/249 [==============================] - 51s 204ms/step
Found 19957 images belonging to 2 classes.
Found 4989 images belonging to 2 classes.
249/249 [==============================] - 85s 340ms/step


In [7]:
batch_extract_test_features()

Found 12500 images belonging to 1 classes.
625/625 [==============================] - 105s 169ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 126s 202ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 108s 173ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 130s 208ms/step
Found 12500 images belonging to 1 classes.
625/625 [==============================] - 215s 343ms/step


In [8]:
if len(models_and_features) == 0:
    models_and_features['vgg16'] = 'vgg16_features.npz'
    models_and_features['vgg19'] = 'vgg19_features.npz'
    models_and_features['resnet50'] = 'resnet50_features.npz'
    models_and_features['inceptionV3'] = 'inception_v3_features.npz'
    models_and_features['xception'] = 'xception_features.npz'

In [9]:
if len(models_and_test_features) == 0:
    models_and_test_features['vgg16'] = 'test_vgg16_features.npz'
    models_and_test_features['vgg19'] = 'test_vgg19_features.npz'
    models_and_test_features['resnet50'] = 'test_resnet50_features.npz'
    models_and_test_features['inceptionV3'] = 'test_inception_v3_features.npz'
    models_and_test_features['xception'] = 'test_xception_features.npz'

In [10]:
def model_train(key, dropout, optimizer, file_header):
    features = np.load(models_and_features[key])
    features_test = np.load(models_and_test_features[key])
    train_features = features['train']
    train_labels = features['train_label'][:len(train_features)]
    valid_features = features['valid']
    valid_labels = features['valid_label'][:len(valid_features)]
    test_features = features_test['test']
    test_filenames = features_test['test_filename']
    
    model = Sequential()
    model.add(Dense(2048, input_shape=(train_features.shape[1],), activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='{0}.hdf5'.format(file_header), verbose=1, save_best_only=True)
    train_result = model.fit(train_features, train_labels, epochs=20, batch_size=common.batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])
    
    with open('{0}_history.pkl'.format(file_header), 'wb') as f:
        pickle.dump(train_result.history, f)
    model.load_weights('{0}.hdf5'.format(file_header))
    prediction = model.predict(test_features, batch_size=common.batch_size)
    prediction = prediction[:, 0].clip(0.01, 0.99)
    test_fileindex = np.array([os.path.splitext(os.path.split(filename)[1])[0] for filename in test_filenames])
    data = np.stack([test_fileindex, prediction], axis=1)
    tmp = pd.DataFrame(data, columns=['id', 'label'])
    tmp['id'] = tmp['id'].apply(pd.to_numeric)
    submit_frame = pd.read_csv('dogs-vs-cats/sample_submission.csv')
    result = pd.merge(submit_frame, tmp, on="id", how='left')
    result = result.rename(index=str, columns={"label_y": "label"})
    result.dropna(axis=0, subset=['label'], inplace=True)
    result[['id','label']].to_csv('{0}_predict.csv'.format(file_header),index=False)

## VGG16

In [12]:
model_train('vgg16', 0.3, Adam(lr=0.001), 'vgg16_Adam_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 15s 763us/step - loss: 1.1056 - acc: 0.9280 - val_loss: 0.5501 - val_acc: 0.9657

Epoch 00001: val_loss improved from inf to 0.55015, saving model to vgg16_Adam_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 12s 618us/step - loss: 2.1497 - acc: 0.8653 - val_loss: 2.1391 - val_acc: 0.8655

Epoch 00002: val_loss did not improve from 0.55015
Epoch 3/20
19940/19940 [==============================] - 12s 618us/step - loss: 2.0468 - acc: 0.8727 - val_loss: 3.0407 - val_acc: 0.8110

Epoch 00003: val_loss did not improve from 0.55015
Epoch 4/20
19940/19940 [==============================] - 12s 620us/step - loss: 1.7937 - acc: 0.8886 - val_loss: 0.4832 - val_acc: 0.9699

Epoch 00004: val_loss improved from 0.55015 to 0.48324, saving model to vgg16_Adam_0.3.hdf5
Epoch 5/20
19940/19940 [==============================] - 12s 622us/step - loss: 2.3155 - acc: 0.8555 - val_l

In [13]:
model_train('vgg16', 0.5, Adam(lr=0.001), 'vgg16_Adam_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 16s 783us/step - loss: 1.2669 - acc: 0.9170 - val_loss: 4.8128 - val_acc: 0.7010

Epoch 00001: val_loss improved from inf to 4.81276, saving model to vgg16_Adam_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 13s 635us/step - loss: 1.3581 - acc: 0.9148 - val_loss: 1.1427 - val_acc: 0.9281

Epoch 00002: val_loss improved from 4.81276 to 1.14274, saving model to vgg16_Adam_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 13s 629us/step - loss: 1.6334 - acc: 0.8983 - val_loss: 0.3730 - val_acc: 0.9767

Epoch 00003: val_loss improved from 1.14274 to 0.37298, saving model to vgg16_Adam_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 13s 642us/step - loss: 0.9200 - acc: 0.9426 - val_loss: 3.2726 - val_acc: 0.7966

Epoch 00004: val_loss did not improve from 0.37298
Epoch 5/20
19940/19940 [==============================] - 13s 635us/

In [14]:
model_train('vgg16', 0.7, Adam(lr=0.001), 'vgg16_Adam_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 16s 778us/step - loss: 1.2481 - acc: 0.9168 - val_loss: 0.7436 - val_acc: 0.9528

Epoch 00001: val_loss improved from inf to 0.74360, saving model to vgg16_Adam_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 12s 627us/step - loss: 1.6046 - acc: 0.8992 - val_loss: 0.6809 - val_acc: 0.9574

Epoch 00002: val_loss improved from 0.74360 to 0.68085, saving model to vgg16_Adam_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 12s 625us/step - loss: 2.1160 - acc: 0.8676 - val_loss: 0.6896 - val_acc: 0.9566

Epoch 00003: val_loss did not improve from 0.68085
Epoch 4/20
19940/19940 [==============================] - 12s 623us/step - loss: 1.3231 - acc: 0.9173 - val_loss: 1.0795 - val_acc: 0.9329

Epoch 00004: val_loss did not improve from 0.68085
Epoch 5/20
19940/19940 [==============================] - 12s 621us/step - loss: 1.1105 - acc: 0.9308 - val_l

In [15]:
model_train('vgg16', 0.3, SGD(lr=0.001), 'vgg16_SGD_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 11s 566us/step - loss: 0.1044 - acc: 0.9598 - val_loss: 0.0420 - val_acc: 0.9823

Epoch 00001: val_loss improved from inf to 0.04196, saving model to vgg16_SGD_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 422us/step - loss: 0.0614 - acc: 0.9783 - val_loss: 0.0409 - val_acc: 0.9829

Epoch 00002: val_loss improved from 0.04196 to 0.04091, saving model to vgg16_SGD_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 8s 418us/step - loss: 0.0513 - acc: 0.9820 - val_loss: 0.0394 - val_acc: 0.9843

Epoch 00003: val_loss improved from 0.04091 to 0.03939, saving model to vgg16_SGD_0.3.hdf5
Epoch 4/20
19940/19940 [==============================] - 8s 419us/step - loss: 0.0485 - acc: 0.9842 - val_loss: 0.0371 - val_acc: 0.9845

Epoch 00004: val_loss improved from 0.03939 to 0.03712, saving model to vgg16_SGD_0.3.hdf5
Epoch 5/20
19940/19940 [==========

In [16]:
model_train('vgg16', 0.5, SGD(lr=0.001), 'vgg16_SGD_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 11s 569us/step - loss: 0.1741 - acc: 0.9444 - val_loss: 0.0481 - val_acc: 0.9807

Epoch 00001: val_loss improved from inf to 0.04812, saving model to vgg16_SGD_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 419us/step - loss: 0.0911 - acc: 0.9707 - val_loss: 0.0436 - val_acc: 0.9823

Epoch 00002: val_loss improved from 0.04812 to 0.04355, saving model to vgg16_SGD_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 8s 421us/step - loss: 0.0813 - acc: 0.9742 - val_loss: 0.0448 - val_acc: 0.9815

Epoch 00003: val_loss did not improve from 0.04355
Epoch 4/20
19940/19940 [==============================] - 8s 422us/step - loss: 0.0702 - acc: 0.9765 - val_loss: 0.0410 - val_acc: 0.9849

Epoch 00004: val_loss improved from 0.04355 to 0.04102, saving model to vgg16_SGD_0.5.hdf5
Epoch 5/20
19940/19940 [==============================] - 8s 417us/step - 

In [17]:
model_train('vgg16', 0.7, SGD(lr=0.001), 'vgg16_SGD_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 11s 567us/step - loss: 0.3390 - acc: 0.9242 - val_loss: 0.0554 - val_acc: 0.9805

Epoch 00001: val_loss improved from inf to 0.05537, saving model to vgg16_SGD_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 419us/step - loss: 0.1540 - acc: 0.9612 - val_loss: 0.0621 - val_acc: 0.9805

Epoch 00002: val_loss did not improve from 0.05537
Epoch 3/20
19940/19940 [==============================] - 8s 417us/step - loss: 0.1241 - acc: 0.9688 - val_loss: 0.0476 - val_acc: 0.9835

Epoch 00003: val_loss improved from 0.05537 to 0.04765, saving model to vgg16_SGD_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 8s 421us/step - loss: 0.1000 - acc: 0.9729 - val_loss: 0.0508 - val_acc: 0.9823

Epoch 00004: val_loss did not improve from 0.04765
Epoch 5/20
19940/19940 [==============================] - 8s 420us/step - loss: 0.0927 - acc: 0.9726 - val_loss: 0

## VGG19

In [19]:
model_train('vgg19', 0.3, Adam(lr=0.001), 'vgg19_Adam_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 16s 779us/step - loss: 1.0290 - acc: 0.9319 - val_loss: 0.6938 - val_acc: 0.9560

Epoch 00001: val_loss improved from inf to 0.69381, saving model to vgg19_Adam_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 12s 625us/step - loss: 1.2855 - acc: 0.9194 - val_loss: 2.9834 - val_acc: 0.8122

Epoch 00002: val_loss did not improve from 0.69381
Epoch 3/20
19940/19940 [==============================] - 12s 623us/step - loss: 1.1024 - acc: 0.9312 - val_loss: 3.0584 - val_acc: 0.8100

Epoch 00003: val_loss did not improve from 0.69381
Epoch 4/20
19940/19940 [==============================] - 12s 619us/step - loss: 1.0682 - acc: 0.9336 - val_loss: 0.8890 - val_acc: 0.9442

Epoch 00004: val_loss did not improve from 0.69381
Epoch 5/20
19940/19940 [==============================] - 12s 619us/step - loss: 1.0852 - acc: 0.9321 - val_loss: 1.2038 - val_acc: 0.9245

Epoch 0000

In [20]:
model_train('vgg19', 0.5, Adam(lr=0.001), 'vgg19_Adam_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 15s 777us/step - loss: 0.9512 - acc: 0.9368 - val_loss: 0.5760 - val_acc: 0.9641

Epoch 00001: val_loss improved from inf to 0.57601, saving model to vgg19_Adam_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 12s 626us/step - loss: 0.9403 - acc: 0.9410 - val_loss: 0.8535 - val_acc: 0.9460

Epoch 00002: val_loss did not improve from 0.57601
Epoch 3/20
19940/19940 [==============================] - 12s 622us/step - loss: 2.3366 - acc: 0.8543 - val_loss: 1.2198 - val_acc: 0.9235

Epoch 00003: val_loss did not improve from 0.57601
Epoch 4/20
19940/19940 [==============================] - 12s 622us/step - loss: 1.0133 - acc: 0.9367 - val_loss: 3.2409 - val_acc: 0.7988

Epoch 00004: val_loss did not improve from 0.57601
Epoch 5/20
19940/19940 [==============================] - 12s 618us/step - loss: 1.6336 - acc: 0.8976 - val_loss: 0.3380 - val_acc: 0.9789

Epoch 0000

In [21]:
model_train('vgg19', 0.7, Adam(lr=0.001), 'vgg19_Adam_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 16s 782us/step - loss: 1.2829 - acc: 0.9151 - val_loss: 0.3653 - val_acc: 0.9765

Epoch 00001: val_loss improved from inf to 0.36531, saving model to vgg19_Adam_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 13s 628us/step - loss: 1.0329 - acc: 0.9350 - val_loss: 0.4245 - val_acc: 0.9735

Epoch 00002: val_loss did not improve from 0.36531
Epoch 3/20
19940/19940 [==============================] - 13s 627us/step - loss: 1.1242 - acc: 0.9294 - val_loss: 1.5406 - val_acc: 0.9042

Epoch 00003: val_loss did not improve from 0.36531
Epoch 4/20
19940/19940 [==============================] - 12s 621us/step - loss: 1.5880 - acc: 0.9010 - val_loss: 1.0299 - val_acc: 0.9357

Epoch 00004: val_loss did not improve from 0.36531
Epoch 5/20
19940/19940 [==============================] - 12s 627us/step - loss: 0.9074 - acc: 0.9433 - val_loss: 0.3344 - val_acc: 0.9791

Epoch 0000

In [22]:
model_train('vgg19', 0.3, SGD(lr=0.001), 'vgg19_SGD_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 12s 581us/step - loss: 0.1046 - acc: 0.9604 - val_loss: 0.0443 - val_acc: 0.9815

Epoch 00001: val_loss improved from inf to 0.04432, saving model to vgg19_SGD_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 419us/step - loss: 0.0598 - acc: 0.9786 - val_loss: 0.0434 - val_acc: 0.9835

Epoch 00002: val_loss improved from 0.04432 to 0.04344, saving model to vgg19_SGD_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 8s 425us/step - loss: 0.0477 - acc: 0.9823 - val_loss: 0.0384 - val_acc: 0.9849

Epoch 00003: val_loss improved from 0.04344 to 0.03839, saving model to vgg19_SGD_0.3.hdf5
Epoch 4/20
19940/19940 [==============================] - 8s 416us/step - loss: 0.0429 - acc: 0.9836 - val_loss: 0.0411 - val_acc: 0.9843

Epoch 00004: val_loss did not improve from 0.03839
Epoch 5/20
19940/19940 [==============================] - 8s 422us/step - 

In [23]:
model_train('vgg19', 0.5, SGD(lr=0.001), 'vgg19_SGD_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 12s 581us/step - loss: 0.1637 - acc: 0.9474 - val_loss: 0.0412 - val_acc: 0.9831

Epoch 00001: val_loss improved from inf to 0.04121, saving model to vgg19_SGD_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 417us/step - loss: 0.0833 - acc: 0.9725 - val_loss: 0.0404 - val_acc: 0.9833

Epoch 00002: val_loss improved from 0.04121 to 0.04042, saving model to vgg19_SGD_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 8s 417us/step - loss: 0.0711 - acc: 0.9767 - val_loss: 0.0382 - val_acc: 0.9855

Epoch 00003: val_loss improved from 0.04042 to 0.03817, saving model to vgg19_SGD_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 8s 418us/step - loss: 0.0671 - acc: 0.9783 - val_loss: 0.0375 - val_acc: 0.9851

Epoch 00004: val_loss improved from 0.03817 to 0.03754, saving model to vgg19_SGD_0.5.hdf5
Epoch 5/20
19940/19940 [==========

In [24]:
model_train('vgg19', 0.7, SGD(lr=0.001), 'vgg19_SGD_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 12s 578us/step - loss: 0.3240 - acc: 0.9247 - val_loss: 0.0437 - val_acc: 0.9853

Epoch 00001: val_loss improved from inf to 0.04366, saving model to vgg19_SGD_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 8s 419us/step - loss: 0.1331 - acc: 0.9659 - val_loss: 0.0522 - val_acc: 0.9829

Epoch 00002: val_loss did not improve from 0.04366
Epoch 3/20
19940/19940 [==============================] - 8s 415us/step - loss: 0.1075 - acc: 0.9715 - val_loss: 0.0459 - val_acc: 0.9837

Epoch 00003: val_loss did not improve from 0.04366
Epoch 4/20
19940/19940 [==============================] - 8s 417us/step - loss: 0.0969 - acc: 0.9737 - val_loss: 0.0411 - val_acc: 0.9851

Epoch 00004: val_loss improved from 0.04366 to 0.04113, saving model to vgg19_SGD_0.7.hdf5
Epoch 5/20
19940/19940 [==============================] - 8s 418us/step - loss: 0.0827 - acc: 0.9741 - val_loss: 0

## ResNet50

In [25]:
model_train('resnet50', 0.3, Adam(lr=0.001), 'resnet50_Adam_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 965us/step - loss: 0.0758 - acc: 0.9777 - val_loss: 0.0338 - val_acc: 0.9876

Epoch 00001: val_loss improved from inf to 0.03375, saving model to resnet50_Adam_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.0383 - acc: 0.9874 - val_loss: 0.0390 - val_acc: 0.9869

Epoch 00002: val_loss did not improve from 0.03375
Epoch 3/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.0321 - acc: 0.9894 - val_loss: 0.0338 - val_acc: 0.9892

Epoch 00003: val_loss did not improve from 0.03375
Epoch 4/20
19940/19940 [==============================] - 16s 796us/step - loss: 0.0324 - acc: 0.9890 - val_loss: 0.0428 - val_acc: 0.9845

Epoch 00004: val_loss did not improve from 0.03375
Epoch 5/20
19940/19940 [==============================] - 16s 796us/step - loss: 0.0286 - acc: 0.9904 - val_loss: 0.0296 - val_acc: 0.9888

Epoch 0

In [26]:
model_train('resnet50', 0.5, Adam(lr=0.001), 'resnet50_Adam_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 960us/step - loss: 0.4638 - acc: 0.9629 - val_loss: 0.2788 - val_acc: 0.9817

Epoch 00001: val_loss improved from inf to 0.27879, saving model to resnet50_Adam_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.5251 - acc: 0.9664 - val_loss: 0.2631 - val_acc: 0.9833

Epoch 00002: val_loss improved from 0.27879 to 0.26308, saving model to resnet50_Adam_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 788us/step - loss: 0.5821 - acc: 0.9631 - val_loss: 0.3410 - val_acc: 0.9785

Epoch 00003: val_loss did not improve from 0.26308
Epoch 4/20
19940/19940 [==============================] - 16s 792us/step - loss: 0.8977 - acc: 0.9436 - val_loss: 0.2609 - val_acc: 0.9837

Epoch 00004: val_loss improved from 0.26308 to 0.26093, saving model to resnet50_Adam_0.5.hdf5
Epoch 5/20
19940/19940 [==============================] - 1

In [27]:
model_train('resnet50', 0.7, Adam(lr=0.001), 'resnet50_Adam_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 964us/step - loss: 0.6103 - acc: 0.9533 - val_loss: 0.3028 - val_acc: 0.9803

Epoch 00001: val_loss improved from inf to 0.30281, saving model to resnet50_Adam_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.7499 - acc: 0.9522 - val_loss: 0.5339 - val_acc: 0.9667

Epoch 00002: val_loss did not improve from 0.30281
Epoch 3/20
19940/19940 [==============================] - 16s 796us/step - loss: 0.8618 - acc: 0.9458 - val_loss: 0.2910 - val_acc: 0.9815

Epoch 00003: val_loss improved from 0.30281 to 0.29096, saving model to resnet50_Adam_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.7650 - acc: 0.9518 - val_loss: 0.6717 - val_acc: 0.9574

Epoch 00004: val_loss did not improve from 0.29096
Epoch 5/20
19940/19940 [==============================] - 16s 793us/step - loss: 1.0788 - acc: 0.9319 -

In [28]:
model_train('resnet50', 0.3, SGD(lr=0.001), 'resnet50_SGD_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 653us/step - loss: 0.1555 - acc: 0.9543 - val_loss: 0.0567 - val_acc: 0.9823

Epoch 00001: val_loss improved from inf to 0.05672, saving model to resnet50_SGD_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 483us/step - loss: 0.0588 - acc: 0.9813 - val_loss: 0.0408 - val_acc: 0.9845

Epoch 00002: val_loss improved from 0.05672 to 0.04082, saving model to resnet50_SGD_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 487us/step - loss: 0.0468 - acc: 0.9835 - val_loss: 0.0351 - val_acc: 0.9851

Epoch 00003: val_loss improved from 0.04082 to 0.03514, saving model to resnet50_SGD_0.3.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 487us/step - loss: 0.0396 - acc: 0.9860 - val_loss: 0.0324 - val_acc: 0.9861

Epoch 00004: val_loss improved from 0.03514 to 0.03242, saving model to resnet50_SGD_0.3.hdf5
Epoch 5/20
19940/19

In [29]:
model_train('resnet50', 0.5, SGD(lr=0.001), 'resnet50_SGD_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 650us/step - loss: 0.1915 - acc: 0.9304 - val_loss: 0.0584 - val_acc: 0.9817

Epoch 00001: val_loss improved from inf to 0.05842, saving model to resnet50_SGD_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 484us/step - loss: 0.0728 - acc: 0.9744 - val_loss: 0.0418 - val_acc: 0.9851

Epoch 00002: val_loss improved from 0.05842 to 0.04179, saving model to resnet50_SGD_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 490us/step - loss: 0.0565 - acc: 0.9805 - val_loss: 0.0379 - val_acc: 0.9861

Epoch 00003: val_loss improved from 0.04179 to 0.03786, saving model to resnet50_SGD_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 484us/step - loss: 0.0505 - acc: 0.9817 - val_loss: 0.0357 - val_acc: 0.9865

Epoch 00004: val_loss improved from 0.03786 to 0.03567, saving model to resnet50_SGD_0.5.hdf5
Epoch 5/20
19940/19

In [30]:
model_train('resnet50', 0.7, SGD(lr=0.001), 'resnet50_SGD_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 658us/step - loss: 0.2710 - acc: 0.8851 - val_loss: 0.0612 - val_acc: 0.9797

Epoch 00001: val_loss improved from inf to 0.06115, saving model to resnet50_SGD_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 490us/step - loss: 0.1047 - acc: 0.9617 - val_loss: 0.0439 - val_acc: 0.9847

Epoch 00002: val_loss improved from 0.06115 to 0.04394, saving model to resnet50_SGD_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 484us/step - loss: 0.0810 - acc: 0.9718 - val_loss: 0.0375 - val_acc: 0.9859

Epoch 00003: val_loss improved from 0.04394 to 0.03753, saving model to resnet50_SGD_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 486us/step - loss: 0.0704 - acc: 0.9736 - val_loss: 0.0346 - val_acc: 0.9871

Epoch 00004: val_loss improved from 0.03753 to 0.03462, saving model to resnet50_SGD_0.7.hdf5
Epoch 5/20
19940/19

## InceptionV3

In [31]:
model_train('inceptionV3', 0.3, Adam(lr=0.001), 'inceptionV3_Adam_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 969us/step - loss: 0.0587 - acc: 0.9853 - val_loss: 0.0175 - val_acc: 0.9944

Epoch 00001: val_loss improved from inf to 0.01752, saving model to inceptionV3_Adam_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 794us/step - loss: 0.0317 - acc: 0.9914 - val_loss: 0.0143 - val_acc: 0.9952

Epoch 00002: val_loss improved from 0.01752 to 0.01427, saving model to inceptionV3_Adam_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 793us/step - loss: 0.0228 - acc: 0.9938 - val_loss: 0.0200 - val_acc: 0.9950

Epoch 00003: val_loss did not improve from 0.01427
Epoch 4/20
19940/19940 [==============================] - 16s 798us/step - loss: 0.0203 - acc: 0.9940 - val_loss: 0.0238 - val_acc: 0.9926

Epoch 00004: val_loss did not improve from 0.01427
Epoch 5/20
19940/19940 [==============================] - 16s 792us/step - loss: 0.0187 - acc: 0.

In [32]:
model_train('inceptionV3', 0.5, Adam(lr=0.001), 'inceptionV3_Adam_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 970us/step - loss: 0.1734 - acc: 0.9795 - val_loss: 0.0708 - val_acc: 0.9946

Epoch 00001: val_loss improved from inf to 0.07084, saving model to inceptionV3_Adam_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 791us/step - loss: 0.3081 - acc: 0.9791 - val_loss: 0.1120 - val_acc: 0.9928

Epoch 00002: val_loss did not improve from 0.07084
Epoch 3/20
19940/19940 [==============================] - 16s 795us/step - loss: 0.2573 - acc: 0.9835 - val_loss: 0.1768 - val_acc: 0.9882

Epoch 00003: val_loss did not improve from 0.07084
Epoch 4/20
19940/19940 [==============================] - 16s 804us/step - loss: 0.2681 - acc: 0.9828 - val_loss: 0.1954 - val_acc: 0.9878

Epoch 00004: val_loss did not improve from 0.07084
Epoch 5/20
19940/19940 [==============================] - 16s 794us/step - loss: 0.2778 - acc: 0.9824 - val_loss: 0.0969 - val_acc: 0.9938

Epoc

In [33]:
model_train('inceptionV3', 0.7, Adam(lr=0.001), 'inceptionV3_Adam_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 19s 973us/step - loss: 0.3572 - acc: 0.9694 - val_loss: 0.1446 - val_acc: 0.9904

Epoch 00001: val_loss improved from inf to 0.14462, saving model to inceptionV3_Adam_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 793us/step - loss: 0.3595 - acc: 0.9765 - val_loss: 0.0909 - val_acc: 0.9936

Epoch 00002: val_loss improved from 0.14462 to 0.09094, saving model to inceptionV3_Adam_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 802us/step - loss: 0.5116 - acc: 0.9677 - val_loss: 0.3230 - val_acc: 0.9797

Epoch 00003: val_loss did not improve from 0.09094
Epoch 4/20
19940/19940 [==============================] - 16s 790us/step - loss: 0.5837 - acc: 0.9633 - val_loss: 0.3763 - val_acc: 0.9763

Epoch 00004: val_loss did not improve from 0.09094
Epoch 5/20
19940/19940 [==============================] - 16s 795us/step - loss: 0.6874 - acc: 0.

In [34]:
model_train('inceptionV3', 0.3, SGD(lr=0.001), 'inceptionV3_SGD_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 667us/step - loss: 0.2245 - acc: 0.9598 - val_loss: 0.0749 - val_acc: 0.9902

Epoch 00001: val_loss improved from inf to 0.07492, saving model to inceptionV3_SGD_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 487us/step - loss: 0.0667 - acc: 0.9889 - val_loss: 0.0422 - val_acc: 0.9924

Epoch 00002: val_loss improved from 0.07492 to 0.04216, saving model to inceptionV3_SGD_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 486us/step - loss: 0.0461 - acc: 0.9903 - val_loss: 0.0322 - val_acc: 0.9928

Epoch 00003: val_loss improved from 0.04216 to 0.03221, saving model to inceptionV3_SGD_0.3.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 492us/step - loss: 0.0360 - acc: 0.9921 - val_loss: 0.0267 - val_acc: 0.9934

Epoch 00004: val_loss improved from 0.03221 to 0.02673, saving model to inceptionV3_SGD_0.3.hdf5
Epoch 5

In [35]:
model_train('inceptionV3', 0.5, SGD(lr=0.001), 'inceptionV3_SGD_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 665us/step - loss: 0.2671 - acc: 0.9223 - val_loss: 0.0792 - val_acc: 0.9902

Epoch 00001: val_loss improved from inf to 0.07920, saving model to inceptionV3_SGD_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 494us/step - loss: 0.0802 - acc: 0.9851 - val_loss: 0.0429 - val_acc: 0.9918

Epoch 00002: val_loss improved from 0.07920 to 0.04293, saving model to inceptionV3_SGD_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 489us/step - loss: 0.0537 - acc: 0.9879 - val_loss: 0.0320 - val_acc: 0.9926

Epoch 00003: val_loss improved from 0.04293 to 0.03201, saving model to inceptionV3_SGD_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 490us/step - loss: 0.0424 - acc: 0.9896 - val_loss: 0.0269 - val_acc: 0.9926

Epoch 00004: val_loss improved from 0.03201 to 0.02688, saving model to inceptionV3_SGD_0.5.hdf5
Epoch 5

In [36]:
model_train('inceptionV3', 0.7, SGD(lr=0.001), 'inceptionV3_SGD_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 13s 676us/step - loss: 0.3318 - acc: 0.8635 - val_loss: 0.0838 - val_acc: 0.9910

Epoch 00001: val_loss improved from inf to 0.08383, saving model to inceptionV3_SGD_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 491us/step - loss: 0.1067 - acc: 0.9743 - val_loss: 0.0424 - val_acc: 0.9914

Epoch 00002: val_loss improved from 0.08383 to 0.04240, saving model to inceptionV3_SGD_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 491us/step - loss: 0.0681 - acc: 0.9839 - val_loss: 0.0313 - val_acc: 0.9922

Epoch 00003: val_loss improved from 0.04240 to 0.03129, saving model to inceptionV3_SGD_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 492us/step - loss: 0.0539 - acc: 0.9853 - val_loss: 0.0264 - val_acc: 0.9922

Epoch 00004: val_loss improved from 0.03129 to 0.02636, saving model to inceptionV3_SGD_0.7.hdf5
Epoch 5

## Xception

In [37]:
model_train('xception', 0.3, Adam(lr=0.001), 'xception_Adam_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 20s 982us/step - loss: 0.0856 - acc: 0.9879 - val_loss: 0.0281 - val_acc: 0.9944

Epoch 00001: val_loss improved from inf to 0.02811, saving model to xception_Adam_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 794us/step - loss: 0.0269 - acc: 0.9945 - val_loss: 0.0205 - val_acc: 0.9962

Epoch 00002: val_loss improved from 0.02811 to 0.02053, saving model to xception_Adam_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 805us/step - loss: 0.0238 - acc: 0.9940 - val_loss: 0.0461 - val_acc: 0.9888

Epoch 00003: val_loss did not improve from 0.02053
Epoch 4/20
19940/19940 [==============================] - 16s 797us/step - loss: 0.0165 - acc: 0.9959 - val_loss: 0.0174 - val_acc: 0.9958

Epoch 00004: val_loss improved from 0.02053 to 0.01742, saving model to xception_Adam_0.3.hdf5
Epoch 5/20
19940/19940 [==============================] - 1

In [38]:
model_train('xception', 0.5, Adam(lr=0.001), 'xception_Adam_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 20s 983us/step - loss: 0.2143 - acc: 0.9831 - val_loss: 0.4623 - val_acc: 0.9701

Epoch 00001: val_loss improved from inf to 0.46230, saving model to xception_Adam_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 801us/step - loss: 0.1850 - acc: 0.9878 - val_loss: 0.2307 - val_acc: 0.9855

Epoch 00002: val_loss improved from 0.46230 to 0.23069, saving model to xception_Adam_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 801us/step - loss: 0.2197 - acc: 0.9858 - val_loss: 0.0955 - val_acc: 0.9940

Epoch 00003: val_loss improved from 0.23069 to 0.09554, saving model to xception_Adam_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 16s 798us/step - loss: 0.1309 - acc: 0.9918 - val_loss: 0.0995 - val_acc: 0.9938

Epoch 00004: val_loss did not improve from 0.09554
Epoch 5/20
19940/19940 [==============================] - 1

In [39]:
model_train('xception', 0.7, Adam(lr=0.001), 'xception_Adam_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 20s 985us/step - loss: 0.2367 - acc: 0.9803 - val_loss: 0.2341 - val_acc: 0.9839

Epoch 00001: val_loss improved from inf to 0.23405, saving model to xception_Adam_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 16s 806us/step - loss: 0.2101 - acc: 0.9865 - val_loss: 0.0964 - val_acc: 0.9938

Epoch 00002: val_loss improved from 0.23405 to 0.09645, saving model to xception_Adam_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 16s 796us/step - loss: 0.2244 - acc: 0.9855 - val_loss: 0.0740 - val_acc: 0.9954

Epoch 00003: val_loss improved from 0.09645 to 0.07402, saving model to xception_Adam_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 16s 798us/step - loss: 0.1847 - acc: 0.9882 - val_loss: 0.1633 - val_acc: 0.9898

Epoch 00004: val_loss did not improve from 0.07402
Epoch 5/20
19940/19940 [==============================] - 1

In [40]:
model_train('xception', 0.3, SGD(lr=0.001), 'xception_SGD_0.3')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 14s 682us/step - loss: 0.3951 - acc: 0.9497 - val_loss: 0.1834 - val_acc: 0.9914

Epoch 00001: val_loss improved from inf to 0.18344, saving model to xception_SGD_0.3.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 492us/step - loss: 0.1288 - acc: 0.9898 - val_loss: 0.0756 - val_acc: 0.9908

Epoch 00002: val_loss improved from 0.18344 to 0.07558, saving model to xception_SGD_0.3.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 494us/step - loss: 0.0677 - acc: 0.9911 - val_loss: 0.0484 - val_acc: 0.9924

Epoch 00003: val_loss improved from 0.07558 to 0.04840, saving model to xception_SGD_0.3.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 491us/step - loss: 0.0482 - acc: 0.9916 - val_loss: 0.0372 - val_acc: 0.9926

Epoch 00004: val_loss improved from 0.04840 to 0.03719, saving model to xception_SGD_0.3.hdf5
Epoch 5/20
19940/19

In [41]:
model_train('xception', 0.5, SGD(lr=0.001), 'xception_SGD_0.5')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 14s 677us/step - loss: 0.4110 - acc: 0.9205 - val_loss: 0.1854 - val_acc: 0.9916

Epoch 00001: val_loss improved from inf to 0.18542, saving model to xception_SGD_0.5.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 493us/step - loss: 0.1412 - acc: 0.9893 - val_loss: 0.0726 - val_acc: 0.9916

Epoch 00002: val_loss improved from 0.18542 to 0.07261, saving model to xception_SGD_0.5.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 495us/step - loss: 0.0740 - acc: 0.9915 - val_loss: 0.0447 - val_acc: 0.9922

Epoch 00003: val_loss improved from 0.07261 to 0.04474, saving model to xception_SGD_0.5.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 492us/step - loss: 0.0514 - acc: 0.9920 - val_loss: 0.0340 - val_acc: 0.9942

Epoch 00004: val_loss improved from 0.04474 to 0.03404, saving model to xception_SGD_0.5.hdf5
Epoch 5/20
19940/19

In [42]:
model_train('xception', 0.7, SGD(lr=0.001), 'xception_SGD_0.7')

Train on 19940 samples, validate on 4980 samples
Epoch 1/20
19940/19940 [==============================] - 14s 679us/step - loss: 0.4569 - acc: 0.8416 - val_loss: 0.2134 - val_acc: 0.9936

Epoch 00001: val_loss improved from inf to 0.21338, saving model to xception_SGD_0.7.hdf5
Epoch 2/20
19940/19940 [==============================] - 10s 496us/step - loss: 0.1812 - acc: 0.9794 - val_loss: 0.0821 - val_acc: 0.9936

Epoch 00002: val_loss improved from 0.21338 to 0.08213, saving model to xception_SGD_0.7.hdf5
Epoch 3/20
19940/19940 [==============================] - 10s 498us/step - loss: 0.0938 - acc: 0.9866 - val_loss: 0.0474 - val_acc: 0.9936

Epoch 00003: val_loss improved from 0.08213 to 0.04740, saving model to xception_SGD_0.7.hdf5
Epoch 4/20
19940/19940 [==============================] - 10s 495us/step - loss: 0.0660 - acc: 0.9886 - val_loss: 0.0350 - val_acc: 0.9944

Epoch 00004: val_loss improved from 0.04740 to 0.03502, saving model to xception_SGD_0.7.hdf5
Epoch 5/20
19940/19